In [1]:
import os
import sys
sys.version_info

sys.version_info(major=3, minor=6, micro=3, releaselevel='final', serial=0)

In [11]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)

import tqdm
import openslide
import cv2 #opencv for image segmentation
import matplotlib.pyplot as plt
import matplotlib

from PIL import Image

0.21.1
1.14.0


* Go through the slides.csv file
* Split into test/validate/train groups 
* using train, apply class labels (10 equal groups of outcomes)
    *    use groups to apply to valid/test
  


In [3]:
np.random.seed(20180502) #so we get the same groupings of train/valid/test
df = pd.read_csv('slides.csv')

# From Olivier to use just tissue type 01
def get_tissue_type(x):
    return x.split('-')[4]

tissue_type = df.slide_file_name.apply(get_tissue_type)
df = df.loc[tissue_type == '01',:].copy()


## equal groups?  is that ok or should it be biased to train?
valid_days_to_death = df['days_to_death'].dropna()
train_groups = np.random.choice(['train','train','valid','test'],len(valid_days_to_death))
df.loc[valid_days_to_death.index,'train_val_test'] = train_groups

In [4]:
cond = df.loc[valid_days_to_death.index,'train_val_test']=='train'

df['class'] =  np.nan
df.loc[df.loc[:,'days_to_death'] < 180,'class'] = 0
df.loc[df.loc[:,'days_to_death'] > 1500,'class'] = 1

In [5]:
df['class'].value_counts()

0.0    41
1.0    33
Name: class, dtype: int64

In [28]:
df.loc[df['class']==0,'train_val_test'].value_counts()


train    19
valid    12
test     10
Name: train_val_test, dtype: int64

Now groups and labels are defined in the dataframe.

Iterate over all the data directories and generate training tree structure:

`
./data/
  /train
    /0
    /1
    ...
    /9
  /valid
    /0
    /1
    ...
    /9
`
We can have different sized images and fast.ai will downsize to 256x256 for us.  We will use this feature.


In [30]:
def get_set_class_dir(filename,base='./'):
    img_class = df.loc[df['slide_file_name'].apply(str.upper)==filename.upper(),'class'].values[0]
    img_group = df.loc[df['slide_file_name'].apply(str.upper)==filename.upper(),'train_val_test'].values[0]
    
    if pd.isnull(img_group):
        img_group='other'
    set_class_dir = '/'.join([img_group,str(img_class)]) + '/'
    return base+set_class_dir

In [7]:
def pull_samples(slide_fn, level=1):
    slide = openslide.open_slide(slide_fn)
    
    keep = []
    empty = []
    bad = []
    tiles = []

    max_empty=0.50
    slide_dim = slide.level_dimensions[level]
    slide_dim_0 = slide.level_dimensions[0]


    slide_width, slide_height = slide_dim
    slide_width_0, slide_height_0 = slide_dim_0

    tile_rgba = np.array(slide.read_region((0,0),level,slide_dim))
    gray = cv2.cvtColor(tile_rgba,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    #plt.imshow(tile_rgba)
    len(range(0, slide_height, 256)), len(range(0, slide_width, 256))

    pct_e = []
    for row in tqdm.tnrange(0, slide_height, 256):
        for col in range(0, slide_width, 256):
            tile_width = min(256, slide_width - col)
            tile_height = min(256, slide_height - row)

            tile_loc_0 = ( col * slide_width_0 // slide_width, row * slide_height_0 // slide_height  )
            tile = slide.read_region(tile_loc_0, level, (tile_width, tile_height))

            thresh_area = thresh[row:(row+tile_height), col:(col+tile_width)]
            area_sum = (thresh_area > -1).sum()
            if area_sum == 0:
                bad.append(tile)
            else:
                pct_empty = (thresh_area == 0).sum()/area_sum
                pct_e.append(pct_empty)
                
                if pct_empty < max_empty: 
                    keep.append(tile)
                else:
                    empty.append(tile)

    slide.close()
    return keep, empty

In [21]:
def tile_slides(slide_array,display=True):
    '''
    Create single 3x3 slide of each subsample
    
    '''
    out_side_length = 3
    num_slides = out_side_length**2

    
    (img_height,img_width) = slide_array[0].size #first image H x W
    total_width  = out_side_length * img_width
    total_height = out_side_length * img_height
    new_im = Image.new('RGB', (total_width, total_height))
    
    slide_sample_idx = np.random.choice(range(len(slide_array)), size=num_slides)

    x_offset,y_offset = (0,0)
#    for i, slide_idx in tqdm.tqdm_notebook(enumerate(slide_sample_idx), total=num_slides):
    for i, slide_idx in enumerate(slide_sample_idx[:num_slides]):

        new_im.paste(slide_array[slide_idx],(x_offset,y_offset))
        x_offset =(x_offset + img_width) % total_width
        if ((i+1) % out_side_length) == 0:
            y_offset += img_height
    if display:
        plt.imshow(new_im)
        
    return new_im

In [29]:
def get_images_put_in_dir(slide_filename,save_path,
                          display=False):
    
    keep,empty = pull_samples(slide_filename)
    for j in range(20):
        tiled_img = tile_slides(keep,display)
        save_name = os.sep.join(path+[file]).split(os.sep)[-1].split('.')[0]
        matplotlib.image.imsave(save_path+f'{save_name}_{j}.png',tiled_img)

In [31]:
BASE = './tmp_tiled/' #base to put in the selected images

counter = 0
for root, dirs, files in os.walk("./data/"):
    path = root.split(os.sep)
    for file in files:
        if file.endswith('.svs') and os.path.basename(file).split('-')[4] == '01':
            directory = get_set_class_dir(file,BASE)
            
            if (not 'nan' in directory) and ('train' in directory or 'valid' in directory):
                print(directory)
                print(os.sep.join(path+[file]))
                if not os.path.exists(directory):
                    os.makedirs(directory)
                get_images_put_in_dir(os.sep.join(path+[file]),directory)
            
#            counter += 1 
#    if counter == 50:
#        break

print(counter)

./tmp_tiled/train/1.0/
./data/58ae7226-5ec1-4f94-a49f-e9fa6893ec96/TCGA-DD-A3A5-11A-01-TS1.7E6F0BBB-FC7E-42F7-9F85-063057B2F417.svs


HBox(children=(IntProgress(value=0, max=22), HTML(value='')))


./tmp_tiled/valid/1.0/
./data/d4dbb21c-2943-436a-a51b-8ead4dfcbdde/TCGA-DD-A1E9-11A-01-TSA.fcc3e0b9-359d-4455-91e9-954e58f28b4e.svs


HBox(children=(IntProgress(value=0, max=16), HTML(value='')))


./tmp_tiled/train/0.0/
./data/bed239b1-8983-41b5-8b9b-56da502201a1/TCGA-KR-A7K0-01A-01-TS1.76635723-DAD6-4012-81CB-709DD7348018.svs


...


./tmp_tiled/valid/0.0/
./data/b830b77d-e5e1-462c-8ef8-3adbc3796845/TCGA-DD-AADM-01A-01-TS1.BFD06147-1675-446B-B48D-800EB504A477.svs


HBox(children=(IntProgress(value=0, max=22), HTML(value='')))


./tmp_tiled/train/1.0/
./data/1824d29c-21df-466e-ae6b-1cd209baa8da/TCGA-DD-A115-01A-01-BS1.0839a9e5-5b00-4722-9b9a-04ea65382018.svs


HBox(children=(IntProgress(value=0, max=24), HTML(value='')))


0
